# What

This has been made because the issues of https://github.com/1jamesthompson1/TAIC-report-summary/issues/236. Has brought to attention that I need to easily be able to look at the logs.

## Modules

In [ ]:
import pandas as pd
import dotenv
from azure.data.tables import TableServiceClient
import os
from io import StringIO

dotenv.load_dotenv(override=True)

# Get the tables

In [ ]:
connection_string = f"AccountName={os.getenv('AZURE_STORAGE_ACCOUNT_NAME')};AccountKey={os.getenv('AZURE_STORAGE_ACCOUNT_KEY')};EndpointSuffix=core.windows.net"
client = TableServiceClient.from_connection_string(conn_str=connection_string)
search_table_client = client.get_table_client(table_name="searchlogs")
results_table_client = client.get_table_client(table_name="resultslogs")
errors_table_client = client.get_table_client(table_name="errorlogs")

In [ ]:
entities = search_table_client.list_entities()
data = [entity for entity in entities]
search_logs = pd.DataFrame(data)
search_logs['start_time'] = pd.to_datetime(search_logs['start_time'], unit='s', utc=True).map(lambda x: x.tz_convert('NZ'))
search_logs.sort_values(by='start_time', ascending=False, inplace=True)

search_logs

In [ ]:
results_logs = pd.DataFrame(results_table_client.list_entities())

results_logs['search_results'] = results_logs['search_results'].map(lambda x: pd.read_json(StringIO(x)))

results_logs

In [ ]:
error_logs = pd.DataFrame(errors_table_client.list_entities())

error_logs

# Merge logs together

In [ ]:
search_logs.set_index(['PartitionKey', 'RowKey'], inplace=True)
results_logs.set_index(['PartitionKey', 'RowKey'], inplace=True)
error_logs.set_index(['PartitionKey', 'RowKey'], inplace=True)

merged_logs = search_logs.join([results_logs, error_logs])

merged_logs = merged_logs[merged_logs.index.get_level_values('PartitionKey') != 'test_user']
merged_logs = merged_logs[merged_logs.index.get_level_values('PartitionKey') != 'Test User']

merged_logs = merged_logs[merged_logs['start_time'] >= pd.Timestamp('2024-07-29 00:00:00', tz='NZ')]
merged_logs

In [ ]:
merged_logs.index.get_level_values(0).value_counts()

## Look at usage in a time period

In [ ]:
import dash
from dash import html, dcc, dash_table
import pandas as pd
from dash.dependencies import Input, Output
df = merged_logs

df = df.loc[df.index.get_level_values('PartitionKey') != 'James Thompson']
df.drop('search_results', axis=1, inplace=True)
df.loc[:, 'user'] = df.index.get_level_values('PartitionKey')

# Convert start_time to timezone-naive values
df.loc[:, 'start_time'] = df['start_time'].dt.tz_localize(None)

app = dash.Dash(__name__)

app.layout = html.Div([
    dcc.DatePickerRange(
        id='date-picker-range',
        start_date=(pd.to_datetime('today') - pd.Timedelta(days=7)).date(),
        end_date=pd.to_datetime('today').date()
    ),
    html.Div(id='user-summary', style={'margin-top': '20px'}),
    html.Div(id='output-container-date-picker-range', style={'margin-top': '20px'})
])

@app.callback(
    [Output('user-summary', 'children'),
     Output('output-container-date-picker-range', 'children')],
    [Input('date-picker-range', 'start_date'),
     Input('date-picker-range', 'end_date')])
def update_output(start_date, end_date):
    if start_date and end_date:
        filtered_df = df[(df['start_time'] >= pd.Timestamp(start_date)) & (df['start_time'] <= pd.Timestamp(end_date))]
        num_users = filtered_df.shape[0]
        user_summary = f'Number of users: {num_users}'
        
        if not filtered_df.empty:
            return (
                user_summary,
                dash_table.DataTable(
                    columns=[{"name": i, "id": i} for i in filtered_df.columns],
                    data=filtered_df.to_dict('records'),
                    style_table={'overflowX': 'auto'},
                    style_cell={'textAlign': 'left', 'maxWidth': '250px', 'whiteSpace': 'normal', 'overflow': 'hidden', 'textOverflow': 'ellipsis'},
                    style_header={
                        'backgroundColor': 'rgb(230, 230, 230)',
                        'fontWeight': 'bold'
                    },
                    style_data_conditional=[
                        {
                            'if': {'row_index': 'odd'},
                            'backgroundColor': 'rgb(248, 248, 248)'
                        }
                    ]
                )
            )
    return 'No data available for selected dates', ''

if __name__ == '__main__':
    app.run_server(debug=True)